In [1]:
import re
import urllib.request
import json
from pymystem3 import Mystem
m = Mystem()

In [3]:
elines = []
with open('nefremova.txt', 'r', encoding='utf-8') as f:
    for line in f:
        elines.append(line)

In [104]:
preffix = []
suffix = []
for i, line in enumerate(elines):
    if '1. префикс' in line:
        if '.\n' not in elines[i-1]:
            preffix.append(elines[i-1].replace('-\n', ''))
    elif '1. суффикс' in line:
        if '.п.' not in elines[i-1]:
            if '(' in elines[i-1]:
                suf = re.findall('.*?\(', elines[i-1])[0]
            else:
                suf = elines[i-1]
            suf = suf.replace('-', '').replace(' ', '').replace('(', '').replace('\n', '')
            if suf not in suff:
                suffix.append(suf)
preffix.extend(['ть', 'ться', 'ся'])
endings = ['ий', 'ый', 'ой', 'ая', 'ое', 'ие', 'а', 'я', 'ы', 'и', 'е', 'у', 'ю']

In [7]:
file_object = open('nests_new1.json', 'r', encoding='utf-8')
json_nests = json.load(file_object)
file_object.close()

In [22]:
def root2rt(root):
    rt = root.replace('\n', '')
    rt = rt.replace('j', '')
    rt = rt.replace('I', '').replace('1', '').replace('2', '').replace('3', '')
    rt = rt.replace('<i>', '').replace('</i>', '')
    rt = rt.replace('(', '').replace(')', '')
    if ' ' in rt:
        rt = re.split(' ', rt)[0]
    elif '/' in rt:
        rt = re.split('/', rt)[-1]
    return rt

In [23]:
idshnik = 0
for n in json_nests:
    for k in n.keys():
        nest = n[k]
        for r in nest:
            if int(r['id']) > idshnik:
                idshnik = int(r['id'])
print(idshnik)

296907


In [24]:
def nprint(nest, root):
    trees ={}
    nr = 0
    for n in nest.values():
        nroot = []
        for l in n:
            if (
                root == l['root'] or
                l['root'].startswith(root + ' ') or
                l['root'].endswith('/' + root)
                ):
                trees[l['root']] = [l['root']]
                
        for r in trees:
            ts = [r]
            for l in n[::-1]:
                for t in ts:
                    if t in l['non-ambiguous'] or t in l['ambiguous']:
                        ts.append(l['root'])
            trees[r] = ts
        
        for r in trees:
            ts = []
            for l in n:
                if l['root'] in ts or l['root'] == r: 
                    for na in l['non-ambiguous']:
                        if na not in ts:
                            ts.append(na)
                    for a in l['ambiguous']:
                        if a not in ts:
                            ts.append(a)
            trees[r].extend(ts)
            
        for r in trees:
            for l in n:
                if l['root'] in trees[r]:
                    string = ''
                    for d in range(l['depth']):
                        string += '\t'
                    string += l['root']
                    print(string)

In [74]:
def add_nod(n, idshnik):

    
    n2 = n
    for e in endings:
        if n2.endswith(e) and len(n2) - len(e) > 2:
            n2 = n2[:(len(n2) - len(e))]


    for jn in json_nests:


        for root in jn:
            rt2 = root2rt(root)
            if len(rt2) > 4:
                for e in endings:
                    if rt2.endswith(e) and len(rt2) - len(e) > 2:
                        rt2 = rt2[:(len(rt2) - len(e))] 
        if rt2 not in n2:
            continue


        for nest in jn.values():
            for j, r in enumerate(nest[::-1]):
                rt = root2rt(r['root'])
                if len(rt) > 4:
                    for e in endings:
                        if rt.endswith(e) and len(rt) - len(e) > 2:
                            rt = rt[:(len(rt) - len(e))]


                if n2.startswith(rt) and n2 != rt:

                    word1 = n
                    mettype1 = 'suf'
                    met1 = n[len(rt) - 1:]
                    ambiguous1 = []
                    non_ambiguous1 = []
                    depth1 = r['depth'] + 1
                    id1 = idshnik
                        
                        
                    try:
                        PoS = m.analyze(word1)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                    except (KeyError, IndexError, BrokenPipeError):
                        PoS = 'unknown'

                    root1 = {'root': word1, 'mettype': mettype1, 'met': met1,
                                  'ambiguous': ambiguous1, 'non-ambiguous': non_ambiguous1, 
                                  'depth': depth1,'id': id1, 'PoS': PoS}

                    for r2 in nest[::-1]:
                        if n in r2['ambiguous']:
                            r['ambiguous'].append(n)
                        elif n in r2['non-ambiguous']:
                            if n not in r['ambiguous']:
                                r['ambiguous'].append(n)
                            r2['ambiguous'].append(n)
                            r2['non-ambiguous'].remove(n)
                    if n not in r['ambiguous']:
                        r['non-ambiguous'].append(n)

                    nest.insert(len(nest) - j, root1) 


                elif rt.endswith(n2) and n2 != rt:
                    if n in ealls:
                        ealls.remove(n)
                    idshnik += 1

                    word1 = n
                    mettype1 = 'pref'
                    met1 = n2[:len(n2) - len(rt)]
                    ambiguous1 = []
                    non_ambiguous1 = []
                    depth1 = r['depth'] + 1
                    id1 = idshnik
                    idshnik += 1
                        
                    try:
                        PoS = m.analyze(word1)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                    except (KeyError, IndexError, BrokenPipeError):
                        PoS = 'unknown'

                    root1 = {'root': word1, 'mettype': mettype1, 'met': met1,
                                  'ambiguous': ambiguous1, 'non-ambiguous': non_ambiguous1, 
                                  'depth': depth1,'id': id1, 'PoS': PoS}

                    for r2 in nest[::-1]:
                        if n in r2['ambiguous']:
                            r['ambiguous'].append(n)
                        elif n in r2['non-ambiguous']:
                            if n not in r['ambiguous']:
                                r['ambiguous'].append(n)
                            r2['ambiguous'].append(n)
                            r2['non-ambiguous'].remove(n)
                    if n not in r['ambiguous']:
                        r['non-ambiguous'].append(n)

                    nest.insert(len(nest) - j, root1) 



In [51]:
def find_word(n):
    for jn in json_nests:
        for nest in jn.values():
            for r in nest:
                if root2rt(r['root']) == root2rt(n):
                    return jn
    return {}

In [87]:
word = 'коньяк'

In [88]:
n = m.lemmatize(word)[0]
nest = find_word(n)
if nest == {}:
    add_nod(n, idshnik)
    nest = find_word(n)
    nprint(nest, n)
else:
    nprint(nest, n)

о
	ко
		кон
		конь
			конь
			коньяк
			коньяк
				коньяк
